In [1]:
# !pip3 install pymysql
import pandas as pd 
import pymysql
pd.set_option('display.max_columns' ,None)

df=pd.read_csv('cleaned_amazon_data.csv')

In [2]:
connect=pymysql.connect(
    host='localhost',
    user='root',
    password='aiktc@1234',
    database='amazon',
    autocommit=True
)

cursor=connect.cursor()
cursor

In [3]:
cursor.execute('USE amazon;')

0

In [4]:
df.columns

Index(['transaction_id', 'order_date', 'customer_id', 'product_id',
       'product_name', 'category', 'subcategory', 'brand',
       'original_price_inr', 'discount_percent', 'discounted_price_inr',
       'quantity', 'subtotal_inr', 'delivery_charges', 'final_amount_inr',
       'customer_city', 'customer_state', 'customer_tier',
       'customer_spending_tier', 'customer_age_group', 'payment_method',
       'delivery_days', 'delivery_type', 'is_prime_member', 'is_festival_sale',
       'customer_rating', 'return_status', 'order_month', 'order_year',
       'order_quarter', 'product_weight_kg', 'is_prime_eligible',
       'product_rating', 'payment_method_simplified'],
      dtype='object')

In [5]:
create_product="""
    CREATE TABLE IF NOT EXISTS products(
        product_id VARCHAR(50) PRIMARY KEY,
        product_name VARCHAR(255),
        category VARCHAR(100),
        subcategory VARCHAR(100),
        brand VARCHAR(100)
    )
"""
cursor.execute(create_product)


0

In [6]:
cursor.execute('show tables;')

4

In [7]:
create_customers = """
CREATE TABLE IF NOT EXISTS customers (
    customer_id VARCHAR(50) PRIMARY KEY,
    customer_city VARCHAR(100),
    customer_state VARCHAR(100),
    customer_tier VARCHAR(20),
    customer_age_group VARCHAR(20)
);
"""
cursor.execute(create_customers)


0

In [8]:
create_time_dimension = """
CREATE TABLE IF NOT EXISTS time_dimension (
    date_id DATE PRIMARY KEY,
    year INT,
    quarter INT,
    month INT,
    day INT,
    weekday VARCHAR(20)
);
"""
cursor.execute(create_time_dimension)



0

In [9]:
cursor.execute("show tables;")
for t in cursor:
    print(t[0])

customers
products
time_dimension
transactions


In [10]:

product_df=df[['product_id', 'product_name', 'category', 'subcategory', 'brand']]

In [11]:
# for _, row in product_df.iterrows():
#     cursor.execute("""
#         INSERT INTO products (product_id, product_name, category, subcategory, brand)
#         VALUES (%s, %s, %s, %s, %s)
#         ON DUPLICATE KEY UPDATE
#             product_name = VALUES(product_name),
#             category = VALUES(category),
#             subcategory = VALUES(subcategory),
#             brand = VALUES(brand);
#     """, tuple(row))


In [12]:
# customer_df=df[['customer_id', 'customer_city', 'customer_state', 'customer_tier','customer_age_group']].drop_duplicates()

# for _,row in customer_df.iterrows():
#     cursor.execute(""" INSERT INTO customers (customer_id, customer_city, customer_state, customer_tier, customer_age_group)
#         VALUES (%s, %s, %s, %s, %s)
#         ON DUPLICATE KEY UPDATE
#             customer_city = VALUES(customer_city),
#             customer_state = VALUES(customer_state),
#             customer_tier = VALUES(customer_tier),
#             customer_age_group = VALUES(customer_age_group)
# """,tuple(row))
    

In [13]:

# df = df.dropna(subset=['order_date'])


# df['order_date'] = pd.to_datetime(df['order_date'], errors='coerce')


# df = df.dropna(subset=['order_date'])

# time_df = df[['order_date']].drop_duplicates().copy()
# time_df['year'] = time_df['order_date'].dt.year
# time_df['quarter'] = time_df['order_date'].dt.quarter
# time_df['month'] = time_df['order_date'].dt.month
# time_df['day'] = time_df['order_date'].dt.day
# time_df['weekday'] = time_df['order_date'].dt.day_name()

# for _, row in time_df.iterrows():
#     cursor.execute("""
#         INSERT INTO time_dimension (date_id, year, quarter, month, day, weekday)
#         VALUES (%s, %s, %s, %s, %s, %s)
#         ON DUPLICATE KEY UPDATE
#             year = VALUES(year),
#             quarter = VALUES(quarter),
#             month = VALUES(month),
#             day = VALUES(day),
#             weekday = VALUES(weekday);
#     """, tuple(row))


In [14]:
create_transactions = """
CREATE TABLE IF NOT EXISTS transactions (
    transaction_id VARCHAR(50) PRIMARY KEY,
    order_date DATE,
    customer_id VARCHAR(50),
    product_id VARCHAR(50),
    product_name VARCHAR(255),
    category VARCHAR(100),
    subcategory VARCHAR(100),
    brand VARCHAR(100),
    original_price_inr DECIMAL(10,2),
    discount_percent DECIMAL(5,2),
    discounted_price_inr DECIMAL(10,2),
    quantity INT,
    subtotal_inr DECIMAL(12,2),
    delivery_charges DECIMAL(10,2),
    final_amount_inr DECIMAL(12,2),
    payment_method VARCHAR(50),
    delivery_days INT,
    delivery_type VARCHAR(50),
    is_prime_member BOOLEAN,
    is_festival_sale BOOLEAN,
    customer_rating DECIMAL(3,2),
    return_status VARCHAR(20),
    order_month INT,
    order_year INT,
    order_quarter INT,
    product_weight_kg DECIMAL(6,3),
    is_prime_eligible BOOLEAN,
    product_rating DECIMAL(3,2),
    payment_method_simplified VARCHAR(50),

    FOREIGN KEY (product_id) REFERENCES products(product_id),
    FOREIGN KEY (customer_id) REFERENCES customers(customer_id),
    FOREIGN KEY (order_date) REFERENCES time_dimension(date_id)
);
"""
cursor.execute(create_transactions)



0

In [15]:
# transaction_columns = [
#     'transaction_id', 'order_date', 'customer_id', 'product_id',
#     'product_name', 'category', 'subcategory', 'brand',
#     'original_price_inr', 'discount_percent', 'discounted_price_inr',
#     'quantity', 'subtotal_inr', 'delivery_charges', 'final_amount_inr',
#     'payment_method', 'delivery_days', 'delivery_type', 'is_prime_member',
#     'is_festival_sale', 'customer_rating', 'return_status',
#     'order_month', 'order_year', 'order_quarter',
#     'product_weight_kg', 'is_prime_eligible', 'product_rating',
#     'payment_method_simplified'
# ]
transaction_df=df[[
    'transaction_id', 'order_date', 'customer_id', 'product_id',
    'product_name', 'category', 'subcategory', 'brand',
    'original_price_inr', 'discount_percent', 'discounted_price_inr',
    'quantity', 'subtotal_inr', 'delivery_charges', 'final_amount_inr',
    'payment_method', 'delivery_days', 'delivery_type', 'is_prime_member',
    'is_festival_sale', 'customer_rating', 'return_status',
    'order_month', 'order_year', 'order_quarter',
    'product_weight_kg', 'is_prime_eligible', 'product_rating',
    'payment_method_simplified'
]]

len(transaction_df.columns)

29

In [16]:
transaction_df['delivery_days'].unique()

array(['3', '6', '4', '5', '2', '1', '7', '2 days', nan, '15', '0'],
      dtype=object)

In [17]:
transaction_df['delivery_days']=transaction_df['delivery_days'].str.replace('2 days','2') 

/var/folders/dq/gcp3r5b912169kz3hx5tbb4r0000gn/T/ipykernel_5536/4050492030.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transaction_df['delivery_days']=transaction_df['delivery_days'].str.replace('2 days','2')


In [18]:
# transaction_df['delivery_days']=transaction_df['delivery_days'].astype(int)

transaction_df['delivery_days']=pd.to_numeric(transaction_df['delivery_days'])

/var/folders/dq/gcp3r5b912169kz3hx5tbb4r0000gn/T/ipykernel_5536/671691234.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transaction_df['delivery_days']=pd.to_numeric(transaction_df['delivery_days'])


In [19]:
transaction_df['delivery_days'].isnull().sum()

np.int64(2223)

In [20]:
# transaction_df['delivery_days'] 

In [21]:
transaction_df['delivery_days'].median()

np.float64(3.0)

In [22]:
transaction_df['delivery_days']=transaction_df['delivery_days'].fillna(transaction_df['delivery_days'].median())

/var/folders/dq/gcp3r5b912169kz3hx5tbb4r0000gn/T/ipykernel_5536/741525133.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transaction_df['delivery_days']=transaction_df['delivery_days'].fillna(transaction_df['delivery_days'].median())


In [23]:
# df['delivery_days'] = pd.to_numeric(df['delivery_days'], errors='coerce').astype('Int64')
transaction_df['delivery_days']=pd.to_numeric(transaction_df['delivery_days'], errors='coerce').astype('Int64') 

/var/folders/dq/gcp3r5b912169kz3hx5tbb4r0000gn/T/ipykernel_5536/2170915260.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transaction_df['delivery_days']=pd.to_numeric(transaction_df['delivery_days'], errors='coerce').astype('Int64')


In [24]:
transaction_df['order_date']=pd.to_datetime(transaction_df['order_date'],errors='coerce')

/var/folders/dq/gcp3r5b912169kz3hx5tbb4r0000gn/T/ipykernel_5536/541228650.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transaction_df['order_date']=pd.to_datetime(transaction_df['order_date'],errors='coerce')


In [25]:
# pd.set_option('display.max_rows' ,None)
transaction_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1081900 entries, 0 to 1081899
Data columns (total 29 columns):
 #   Column                     Non-Null Count    Dtype         
---  ------                     --------------    -----         
 0   transaction_id             1081900 non-null  object        
 1   order_date                 978165 non-null   datetime64[ns]
 2   customer_id                1081900 non-null  object        
 3   product_id                 1081900 non-null  object        
 4   product_name               1081900 non-null  object        
 5   category                   1081900 non-null  object        
 6   subcategory                1081900 non-null  object        
 7   brand                      1081900 non-null  object        
 8   original_price_inr         1081900 non-null  float64       
 9   discount_percent           1081900 non-null  float64       
 10  discounted_price_inr       1081900 non-null  float64       
 11  quantity                   1081900 no

In [26]:
transaction_df.columns

Index(['transaction_id', 'order_date', 'customer_id', 'product_id',
       'product_name', 'category', 'subcategory', 'brand',
       'original_price_inr', 'discount_percent', 'discounted_price_inr',
       'quantity', 'subtotal_inr', 'delivery_charges', 'final_amount_inr',
       'payment_method', 'delivery_days', 'delivery_type', 'is_prime_member',
       'is_festival_sale', 'customer_rating', 'return_status', 'order_month',
       'order_year', 'order_quarter', 'product_weight_kg', 'is_prime_eligible',
       'product_rating', 'payment_method_simplified'],
      dtype='object')

In [ ]:
# # 1️⃣ Drop rows where order_date is NaT
# transaction_df = transaction_df.dropna(subset=['order_date'])

# # 2️⃣ Convert columns to MySQL-friendly types
# transaction_df['order_date'] = transaction_df['order_date'].dt.date  # datetime.date
# transaction_df['is_prime_member'] = transaction_df['is_prime_member'].astype(bool)
# transaction_df['is_festival_sale'] = transaction_df['is_festival_sale'].astype(bool)
# transaction_df['is_prime_eligible'] = transaction_df['is_prime_eligible'].astype(bool)

# # 3️⃣ Prepare the insert query
# insert_query = """
# INSERT INTO transactions (
#     transaction_id, order_date, customer_id, product_id,
#     product_name, category, subcategory, brand,
#     original_price_inr, discount_percent, discounted_price_inr,
#     quantity, subtotal_inr, delivery_charges, final_amount_inr,
#     payment_method, delivery_days, delivery_type, is_prime_member,
#     is_festival_sale, customer_rating, return_status,
#     order_month, order_year, order_quarter,
#     product_weight_kg, is_prime_eligible, product_rating,
#     payment_method_simplified
# ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,
#           %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
# ON DUPLICATE KEY UPDATE
#     final_amount_inr = VALUES(final_amount_inr),
#     customer_rating = VALUES(customer_rating),
#     product_rating = VALUES(product_rating);
# """

# # 4️⃣ Convert DataFrame to list of tuples for executemany
# data_tuples = [tuple(x) for x in transaction_df.to_numpy()]

# # 5️⃣ Execute the insert in bulk
# cursor.executemany(insert_query, data_tuples)

978165